In [46]:
import dask.dataframe as dd
from dask_ml.model_selection import train_test_split
from typing import Optional
import os
import dask

In [2]:
cd ../

/app


/home/wslvenug/venv/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
!pwd

/app


In [4]:
def split_dataset(df:dd.core.DataFrame, test_size: float, stratify_column:Optional[str]=None)-> tuple[dd.core.DataFrame, dd.core.DataFrame]:
        if stratify_column is None:
            return train_test_split(df, test_size=test_size, random_state=1234, shuffle=True)
        unique_column_values = df[stratify_column].unique()  
        first_dfs = []
        second_dfs = []
        for unique_set_value in unique_column_values:
            sub_df = df[df[stratify_column] == unique_set_value]
            sub_first_df, sub_second_df = train_test_split(sub_df, test_size=test_size, random_state=1234,shuffle=True)          
            first_dfs.append(sub_first_df)
            second_dfs.append(sub_second_df)
        first_df = dd.concat(first_dfs)
        second_df = dd.concat(second_dfs)
        return first_df, second_df

In [5]:
train_df = dd.read_csv("./data/raw/ghc/ghc_train.tsv", sep="\t", header=0)

In [6]:
test_df = dd.read_csv("./data/raw/ghc/ghc_test.tsv", sep="\t", header=0)

In [7]:
print(train_df.size)

<dask_expr.expr.Scalar: expr=ReadCSV(28dabe4).size(), dtype=int64>


In [8]:
print(test_df.size)

<dask_expr.expr.Scalar: expr=ReadCSV(79b189f).size(), dtype=int64>


In [9]:
train_df.head(10)

,text,hd,cv,vo
0,He most likely converted to islam due to his n...,0,0,0
1,So Ford lied about being a psychologist. Recor...,0,0,0
2,Jobs. Education. Ending abuse of Nation. CA43.,0,0,0
3,"I share a lot of your values, & like many who ...",0,0,0
4,I am so ready to get back to blogging! www.ben...,0,0,0
5,taking a look at new opportunity called FX Pro...,0,0,0
6,Reflecting back when I was in school with Spec...,0,0,0
7,"Let's be honest everyone, last year there were...",0,0,0
8,2007 Nuke plant in Syria,0,0,0
9,NBC's Chuck Todd Thinks He's Figured It All Ou...,0,0,0


In [10]:
test_df.head(10)

,text,hd,cv,vo
0,https://www.youtube.com/watch?v=kACWpKAKtak A ...,0,0,0
1,Very nice! I tend to get tired of the constant...,0,0,0
2,Watch today. https://circumcisionmovie.com/,0,0,0
3,""" Thinking Venues "" First Color Layer blocking...",0,0,0
4,What about death penalty for perpetrators and...,0,0,0
5,You have insecurities? Fuck you,0,0,0
6,Powerful Truths About Donald Trump the Media D...,0,0,0
7,Just saw an Oreo cookie commercial. Drool.,0,0,0
8,I bet a neural net with half a dozen nodes co...,0,0,0
9,"Country by country, the 'Prague Spring' that w...",0,0,0


In [11]:
train_df["label"] = (train_df["hd"] + train_df["cv"] + train_df["vo"] > 0).astype(int)
test_df["label"] = (test_df["hd"] + test_df["cv"] + test_df["vo"] > 0).astype(int)

In [12]:
train_df.head(10)

,text,hd,cv,vo,label
0,He most likely converted to islam due to his n...,0,0,0,0
1,So Ford lied about being a psychologist. Recor...,0,0,0,0
2,Jobs. Education. Ending abuse of Nation. CA43.,0,0,0,0
3,"I share a lot of your values, & like many who ...",0,0,0,0
4,I am so ready to get back to blogging! www.ben...,0,0,0,0
5,taking a look at new opportunity called FX Pro...,0,0,0,0
6,Reflecting back when I was in school with Spec...,0,0,0,0
7,"Let's be honest everyone, last year there were...",0,0,0,0
8,2007 Nuke plant in Syria,0,0,0,0
9,NBC's Chuck Todd Thinks He's Figured It All Ou...,0,0,0,0


In [13]:
test_df.head(10)

,text,hd,cv,vo,label
0,https://www.youtube.com/watch?v=kACWpKAKtak A ...,0,0,0,0
1,Very nice! I tend to get tired of the constant...,0,0,0,0
2,Watch today. https://circumcisionmovie.com/,0,0,0,0
3,""" Thinking Venues "" First Color Layer blocking...",0,0,0,0
4,What about death penalty for perpetrators and...,0,0,0,0
5,You have insecurities? Fuck you,0,0,0,0
6,Powerful Truths About Donald Trump the Media D...,0,0,0,0
7,Just saw an Oreo cookie commercial. Drool.,0,0,0,0
8,I bet a neural net with half a dozen nodes co...,0,0,0,0
9,"Country by country, the 'Prague Spring' that w...",0,0,0,0


In [14]:
train_df, dev_df = split_dataset(train_df, 0.3,stratify_column="label")

In [15]:
train_df.head(10)

,text,hd,cv,vo,label
13501,Boris Johnson has written another storming art...,0,0,0,0
14241,@Tripper,0,0,0,0
2001,Thanks Ken! And thanks always for your photogr...,0,0,0,0
15944,First cousin marriages do not always produce g...,0,0,0,0
5218,WATCH: Texas Woman Flees with Baby in Carrier ...,0,0,0,0
7452,POLL: Should I run for President AGAIN?,0,0,0,0
10360,"WINNING! Student Wins $25,000 After Suspension...",0,0,0,0
5280,"It’s not racist what so ever to want cleaner, ...",0,0,0,0
1914,if you have been censored by twitter it makes ...,0,0,0,0
18110,#ReleaseTheMemo #FISAmisconduct,0,0,0,0


In [16]:
test_df.head(10)

,text,hd,cv,vo,label
0,https://www.youtube.com/watch?v=kACWpKAKtak A ...,0,0,0,0
1,Very nice! I tend to get tired of the constant...,0,0,0,0
2,Watch today. https://circumcisionmovie.com/,0,0,0,0
3,""" Thinking Venues "" First Color Layer blocking...",0,0,0,0
4,What about death penalty for perpetrators and...,0,0,0,0
5,You have insecurities? Fuck you,0,0,0,0
6,Powerful Truths About Donald Trump the Media D...,0,0,0,0
7,Just saw an Oreo cookie commercial. Drool.,0,0,0,0
8,I bet a neural net with half a dozen nodes co...,0,0,0,0
9,"Country by country, the 'Prague Spring' that w...",0,0,0,0


In [18]:
dev_df.head(10)

,text,hd,cv,vo,label
20212,There's an awesome programme on Ch5 right now ...,0,0,0,0
542,"Who changed the truth of God into a lie, and w...",0,0,0,0
20934,"Saw Dunkirk, yet another Nolan film. He likes ...",0,0,0,0
7788,A little late night something something from a...,0,0,0,0
20281,"LPT: When starting a new job, request digital ...",0,0,0,0
17712,Try the #NordicFrontier for soothing #National...,0,0,0,0
21727,"To celebrate our 37th Wedding Anniversary, we ...",0,0,0,0
2182,Why define people by their sexual inclination ...,0,0,0,0
21521,“At the end fiat money returns to its inner va...,0,0,0,0
12413,agreed but mainly muzzies shouldn't.,0,0,0,0


In [19]:
def assign_split_names_to_data_frames_and_merge(
                            train_df: dd.core.DataFrame,
                            dev_df: dd.core.DataFrame,
                            test_df: dd.core.DataFrame
                            )-> dd.core.DataFrame:
    train_df["split"] = "train"
    dev_df["split"] = "dev"
    test_df["split"] = "test"
    return dd.concat([train_df,dev_df,test_df])

In [20]:
df = assign_split_names_to_data_frames_and_merge(train_df, dev_df, test_df)

In [21]:
df.head(10)

,text,hd,cv,vo,label,split
13501,Boris Johnson has written another storming art...,0,0,0,0,train
14241,@Tripper,0,0,0,0,train
2001,Thanks Ken! And thanks always for your photogr...,0,0,0,0,train
15944,First cousin marriages do not always produce g...,0,0,0,0,train
5218,WATCH: Texas Woman Flees with Baby in Carrier ...,0,0,0,0,train
7452,POLL: Should I run for President AGAIN?,0,0,0,0,train
10360,"WINNING! Student Wins $25,000 After Suspension...",0,0,0,0,train
5280,"It’s not racist what so ever to want cleaner, ...",0,0,0,0,train
1914,if you have been censored by twitter it makes ...,0,0,0,0,train
18110,#ReleaseTheMemo #FISAmisconduct,0,0,0,0,train


In [22]:
df["dataset_name"] = "ghc"

In [23]:
df.head(10)

,text,hd,cv,vo,label,split,dataset_name
13501,Boris Johnson has written another storming art...,0,0,0,0,train,ghc
14241,@Tripper,0,0,0,0,train,ghc
2001,Thanks Ken! And thanks always for your photogr...,0,0,0,0,train,ghc
15944,First cousin marriages do not always produce g...,0,0,0,0,train,ghc
5218,WATCH: Texas Woman Flees with Baby in Carrier ...,0,0,0,0,train,ghc
7452,POLL: Should I run for President AGAIN?,0,0,0,0,train,ghc
10360,"WINNING! Student Wins $25,000 After Suspension...",0,0,0,0,train,ghc
5280,"It’s not racist what so ever to want cleaner, ...",0,0,0,0,train,ghc
1914,if you have been censored by twitter it makes ...,0,0,0,0,train,ghc
18110,#ReleaseTheMemo #FISAmisconduct,0,0,0,0,train,ghc


In [24]:
required_columns = {"text", "label", "split", "dataset_name"} 
split_names = {"train", "dev", "test"}
if any(required_column not in df.columns.values for required_column in required_columns):
    raise ValueError(f"Dataset must contain all required columns: {required_columns}")


In [45]:
len(df["split"])

27546

In [37]:
df["split"].tail(5)

5505    test
5506    test
5507    test
5508    test
5509    test
Name: split, dtype: object

In [79]:
#unique_split_names = set(df["split"].unique()) #.tolist())
unique = {
    name: df[name].unique()
    for name in df.columns if name=="split"
}
#unique_split_names = df.split.unique()


In [80]:
# traverse=True is default, but being explicit that we are traversing the dict for dask objects
unique_split_names = dask.compute(unique, traverse=True)[0]

In [90]:
list(unique_split_names["split"].unique())

['train', 'test', 'dev']

In [97]:
unique_split_names = list(dask.compute(df["split"].unique())[0])

In [98]:
print(unique_split_names)
print(split_names)

['train', 'test', 'dev']
{'train', 'dev', 'test'}


In [100]:
sorted(unique_split_names) == sorted(split_names)



True

In [ ]:
if unique_split_names != self.split_names:
    raise ValueError(f"Dataset must contain all required split names: {self.split_names}")